In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Celiac_Disease/GSE112102'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"First Degree Relatives (FDRs) of Patients with Celiac Disease Harbour an Intestinal Transcriptomic Signature that Might Protect Them from Enterocyte Damage"
!Series_summary	"Patients with Celiac Disease, first degree relatives of celiac patients and control groups displayed significant differential gene expression."
!Series_overall_design	"All subjects were underwent upper GI endoscopic examination. Multiple mucosal biopsies were obtained using spike biopsy forceps. Four bits of tissues in 10% formal saline were sent for histological assessment for villous architecture assessment. Other 4-6 bites of mucosal biopsies were taken in RNA later for gene expression analysis. In this study, FDRs were selected randomly and none of the FDRs were related to Celiac patients."
Sample Characteristics Dictionary:
{0: ['tissue: intestinal mucus'], 1: ['group: CeD', 'group: control', 'group: FDR'], 2: ['age: 40', 'age: 21', 'age: 23', 'age: 17', 'age: 19', 'age: 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check gene expression data availability
# Based on the summary in STEP 1, it mentions "gene expression analysis."
is_gene_available = True

# Check data availability for 'Celiac_Disease'
trait_row = 1  # 'Celiac_Disease' is part of the group key

def convert_trait(value):
    # Extract the actual value after the colon
    key, val = value.split(': ', 1)
    # Convert to binary: 0 for control, 1 for CeD, None for others like FDR
    if val == "CeD":
        return 1
    elif val == "control":
        return 0
    else:
        return None

# Check data availability for 'age'
age_row = 2  # 'age' is part of the key

def convert_age(value):
    # Extract the actual value after the colon
    key, val = value.split(': ', 1)
    try:
        return float(val)
    except ValueError:
        return None

# Check data availability for 'gender'
gender_row = 4  # 'gender' is part of the key

def convert_gender(value):
    # Extract the actual value after the colon
    key, val = value.split(': ', 1)
    if val == "Male":
        return 1
    elif val == "Female":
        return 0
    else:
        return None

# Save cohort information
save_cohort_info('GSE112102', './preprocessed/Celiac_Disease/cohort_info.json', is_gene_available, trait_row is not None)

# Now, if trait_row is not None, extract clinical data, else skip this step.
selected_clinical_data = geo_select_clinical_features(clinical_data, 'Celiac_Disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
csv_path = './preprocessed/Celiac_Disease/trait_data/GSE112102.csv'
selected_clinical_data.to_csv(csv_path)
print(preview_df(selected_clinical_data))


{'GSM3057759': [1.0, 40.0, 1.0], 'GSM3057760': [1.0, 21.0, 1.0], 'GSM3057761': [1.0, 23.0, 1.0], 'GSM3057762': [1.0, 21.0, 0.0], 'GSM3057763': [1.0, 23.0, 0.0], 'GSM3057764': [1.0, 17.0, 0.0], 'GSM3057765': [1.0, 19.0, 1.0], 'GSM3057766': [1.0, 21.0, 1.0], 'GSM3057767': [1.0, 18.0, 0.0], 'GSM3057768': [1.0, 20.0, 1.0], 'GSM3057769': [1.0, 19.0, 0.0], 'GSM3057770': [1.0, 32.0, 1.0], 'GSM3057771': [0.0, 38.0, 1.0], 'GSM3057772': [0.0, 42.0, 1.0], 'GSM3057773': [0.0, 27.0, 1.0], 'GSM3057774': [0.0, 27.0, 1.0], 'GSM3057775': [0.0, 21.0, 1.0], 'GSM3057776': [0.0, 28.0, 1.0], 'GSM3057777': [0.0, 27.0, 1.0], 'GSM3057778': [0.0, 32.0, 0.0], 'GSM3057779': [0.0, 26.0, 1.0], 'GSM3057780': [0.0, 23.0, 1.0], 'GSM3057781': [0.0, 28.0, 0.0], 'GSM3057782': [nan, 25.0, 1.0], 'GSM3057783': [nan, 32.0, 0.0], 'GSM3057784': [nan, 32.0, 0.0], 'GSM3057785': [nan, 17.0, 1.0], 'GSM3057786': [nan, 42.0, 1.0], 'GSM3057787': [nan, 35.0, 1.0], 'GSM3057788': [nan, 32.0, 0.0], 'GSM3057789': [nan, 18.0, 0.0], 'GSM305

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651237', 'ILMN_1651254', 'ILMN_1651262', 'ILMN_1651278',
       'ILMN_1651282', 'ILMN_1651285', 'ILMN_1651315', 'ILMN_1651336',
       'ILMN_1651346', 'ILMN_1651347', 'ILMN_1651364', 'ILMN_1651378',
       'ILMN_1651385', 'ILMN_1651405', 'ILMN_1651415', 'ILMN_1651429'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the appropriate keys from the annotation dictionary.
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to convert the gene data
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Celiac_Disease/gene_data/GSE112102.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Celiac_Disease')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE112102', './preprocessed/Celiac_Disease/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Celiac_Disease/GSE112102.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Celiac_Disease', the least common label is '0.0' with 11 occurrences. This represents 47.83% of the dataset.
The distribution of the feature 'Celiac_Disease' in this dataset is fine.

Quartiles for 'Age':
  25%: 21.0
  50% (Median): 23.0
  75%: 28.0
Min: 17.0
Max: 42.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 7 occurrences. This represents 30.43% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

